# Part 1 - Textual Data

In this workshop, we will provide an introduction to working with the various types of hydrological data you are likely to encounter.

The learning objectives are to:

- Learn how to load various types of data from object storage ("the cloud")
- Learn about the different types of textual file formats that are commonly used and how to work with them in Python using Pandas and Numpy
- Learn about the different types of spatial file formats that are commonly used and how to work with them in Python using GeoPandas
- Learn about the new Zarr format for multi-dimensional large datasets and how to work with it in Python using Xarray
- Learn how to utilise Matplotlib and Cartopy's plotting libraries to quickly visualise data

We will be teaching via Jupyter notebooks - which you are hopefully reading this from. We will be going through the notebooks cell-by-cell together. Code cells can be run individually by clicking on the cell and using ```Shift+Enter```.

There will be three notebooks covering, respectively, textual data, vector data, gridded or multi-dimensional data.


## Textual data

In this, first, notebook we will focus on textual, human-readable data.

Before getting started we need to install and import the packages that we will be using in the notebook.

In [ ]:
## ONLY NEEDS TO BE RUN IF USING GOOGLE COLABS
%%capture
!pip install s3fs

In [ ]:
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import s3fs
from IPython.display import Image

We will be using data stored in an object storage bucket through out the session (see the [Data Access notebook](https://github.com/eugmag/ghana_training_2026/blob/main/Day1_Hydrological-Datasets/Ghana_Remote_Access_training.ipynb) for a reminder of how to make use of remote storage):

In [ ]:
s3 = s3fs.S3FileSystem(anon=True, client_kwargs={'endpoint_url': "https://fdri-o.s3-ext.jc.rl.ac.uk"})
s3.ls('s3://example-data/')

Often referred to as '[ASCII](https://en.wikipedia.org/wiki/ASCII)', textual data is data that is human-readable, this means that you can open it with a text editor and see the actual data written down. This differs from binary data formats - like NetCDF, Zarr, Shapefiles - which are stored directly as specifically laid-out binary digits - 0s and 1s - which only computers can read directly. Exactly how the 0s and 1s are laid out and exactly how they represent/translate to the data we are interested in (the encoding) defines the file format. They need special programmes that understand the layout and encoding in order to be read in and processed.

Textual data is considered by many to be easier to work with as all you need is a text editor to get started. Data that is typically stored as ASCII includes gridded outputs from some hydrological models and rain-gauge data.

We will be using [Pandas](https://pandas.pydata.org/), a powerful python library for working with tabulated data, which is what you usually get in textual data files. We will explore some of what you can do with Pandas here, but there is a huge amount of functionality we won't cover that is documented on the Pandas website.

We will also be reading in data that is stored remotely on object storage instead of locally on disk.

Let's start with some simple textual river flow data, such as that which might be output by a hydrological model:

In [ ]:
obsdata = pd.read_csv('s3://example-data/obsflows.csv', storage_options={'endpoint_url': "https://fdri-o.s3-ext.jc.rl.ac.uk", 'anon': True})

obsdata

This is **csv** data. CSV stands for "[Comma Separated Values](https://en.wikipedia.org/wiki/Comma-separated_values)" and means that if you were to open this data up in a text editor you would see the values separated by commas. In this case the commas are the "delimeters" that separate or "delimit" the individual values from each other. Even though CSV stands for **Comma** Separated Values, you may also see other characters being used as delimiters in CSV files, the most common alternative is tabs, which show up as gaps between the numbers in a text editor.

Taking a look at the data table now, we can see that we have rows representing the date and it's not immediately clear what the columns represent. The column headers are in fact catchment IDs, a 5-digit number given to all catchments in the UK, with the first two numbers representing the river-basin the catchment ultimately drains into and the last 3 numbers are zero-padded and are representing the sub-catchment within this river basin. So 39001 translates as 'river basin number 39' (which is the Thames) and sub-catchment number '01' (which in this case is the Thames as far as Kingston). More information on how catchments are identified here: https://nrfa.ceh.ac.uk/data/about-data/catchment-information

This is flow data, so each value in the table will be a flow amount in $m^3/day$. We can see straight away that there are a lot of -1 values. This is likely missing data.

We can also see that the date information is split over three columns (day, month, and year). Pandas has some excellent date-time handling capabilities, but it sometimes needs to be told how to interpret the date-time information in the data first before this can be unlocked.

We can use pandas to read in the CSV again and tell it which columns to convert to a date.

In [ ]:
%%capture
obsdata = pd.read_csv('s3://example-data/obsflows.csv', # read in the csv
                      parse_dates={'Times': [0,1,2]}, dayfirst=True,  # use the first three colums to create the date
                      storage_options={'endpoint_url': "https://fdri-o.s3-ext.jc.rl.ac.uk", 'anon': True})
obsdata.set_index('Times', inplace=True) # set the "Times" column as index

The first line is where we read in the data, note we have now added in an extra argument telling pandas how to handle the date-time information in the data.

The second line sets the new 'Times' column that we have created from the original 'day', 'month', and 'year' columns as the Index, which is what Pandas uses to work with the data.

In [ ]:
obsdata

Let's now have a look at one of the catchments in detail:

We can cleverly select out the date range and catchment we wish to plot using the row and column names. As the row names (or the 'Index' in Pandas parlance) contains labels for each row in YYYY-MM-DD format, we can use these names to select out individual or ranges of rows as ```dataset.loc[rowname, colname]``` or ```dataset.loc[startrowname:endrowname, startcolname:endcolname]```

In [ ]:
obsplot = obsdata.loc['2000-01-01':'2015-12-31', '39010']

Pandas integrates with the [matplotlib](https://matplotlib.org/) plotting library directly, and you can just call ```.plot()``` on a dataframe to get a simple visual representation of the data:

In [ ]:
obsplot.plot()

If we wish to modify the plot, this is also possible:

In [ ]:
plt.rcParams.update({'font.size': 10}) # make the text in the plot easier to read
ax = plt.gca() # gca = get current axis, allows us to get the most recent axis used...
obsplot.plot(ax=ax, label = 'obsflows') # ... and specify it to the plotting command, which is simply dataframename.plot()
ax.set_ylim([0, 25]) # set the y axis lower and upper bounds
plt.title('39010') # set the title
plt.ylabel('Daily mean flow (' + r'$m^3$' + ')') # set the y axis label
plt.legend() # add a legend

Some of the data points look a bit unusual, suddenly going very low.

It's likely the '-1' values we could see in the table earlier. We can deal with these by telling pandas that this value represents a missing value, so that it ignores them when plotting them and doing any other analysis.

We can tell pandas how to handle this by adding another argument to the ```.read_csv(...)``` function.

In [ ]:
%%capture
obsdata = pd.read_csv('s3://example-data/obsflows.csv', # read in the csv
                      parse_dates={'Times': [0,1,2]}, dayfirst=True, # use the first three colums to create the date
                      na_values = -1, # tell pandas that -1 represents a missing value
                      storage_options={'endpoint_url': "https://fdri-o.s3-ext.jc.rl.ac.uk", 'anon': True})
obsdata.set_index('Times', inplace=True) # set the "Times" column as index

In [ ]:
obsdata

We can see from the table above the **-1** values have been replace with **NaN**.

Let's try the plotting again:

In [ ]:
# timeperiod and gauge ID to plot
startplot = '2000-01-01'
endplot   = '2015-12-31'
catchmentplot = '39010'

obsplot = obsdata[startplot:endplot]

plt.rcParams.update({'font.size': 10})
ax = plt.gca()
obsplot[catchmentplot].plot(ax=ax, label = 'obsflows')
ylims = ax.get_ylim()
ax.set_ylim([0, ylims[1]])
plt.title(catchmentplot)
plt.ylabel('Daily mean flow (' + r'$m^3$' + ')')
plt.legend()

Much better! What pandas has done is simply leave a blank space where the data is NaN, which would show up if you zoomed in the plot a bit more. We can zoom in on the plot by using the same plotting code as before but with a smaller time period:

In [ ]:
# timeperiod and gauge ID to plot
startplot = '2010-01-01'
endplot   = '2010-05-31'
catchmentplot = '39010'

obsplot = obsdata[startplot:endplot]

plt.rcParams.update({'font.size': 10})
ax = plt.gca()
obsplot[catchmentplot].plot(ax=ax, label = 'obsflows')
ylims = ax.get_ylim()
ax.set_ylim([0, ylims[1]])
plt.title(catchmentplot)
plt.ylabel('Daily mean flow (' + r'$m^3$' + ')')
plt.legend()

We can see that there is a small gap in the plot, indicating that pandas is correctly recognising the missing values!

The same plotting code can be used for other catchments and time periods

Use the snippet below to try plotting different catchments and time periods from the table.

Some additional interesting catchment IDs:
- 39001 (Thames at Kingston)
- 39020 (Coln at Bibury)
- 81002 (Cree at Newton Stewart)

Are there any differences between the flow patterns?

The UK experienced notable hydrological events in 1975-1976 and 2010-2012. Can you plot these events?

In [ ]:
# timeperiod and gauge ID to plot
startplot = '2005-01-01'
endplot   = '2008-12-31'
catchmentplot = '39020'

obsplot = obsdata[startplot:endplot]

plt.rcParams.update({'font.size': 10})
ax = plt.gca()
obsplot[catchmentplot].plot(ax=ax, label = 'obsflows')
ylims = ax.get_ylim()
ax.set_ylim([0, ylims[1]])
plt.title(catchmentplot)
plt.ylabel('Daily mean flow (' + r'$m^3$' + ')')
plt.legend()

# Supplementary

Let's have a look at annual averages for a given catchments to see if we can pick out any particularly wet/dry years.

To check if Pandas has recognised the information in the Times column as date-time information we can check its type. We can access the information in this column directly by using the 'index' accessor, as we set this column as the index earlier:

In [ ]:
type(obsdata.index)

'DatetimeIndex' is showing up, which means that Pandas has indeed recognised it as a special category of data - date and time information.

We can therefore pull out some useful information from it, such as the year, month, day, but also less obvious information such as day-of-week, quarter/season, days-in-month...

In [ ]:
obsdata.index.year

We can use this information to group the rows of the table into categories, and then perform some operation on these groups. For example, to find the yearly mean for each catchment:

In [ ]:
annavgs = obsdata.groupby(obsdata.index.year).mean()

annavgs

Note that the default behaviour of arithmetic operations (such as mean, sum etc.) applied to pandas dataframes where there is missing data is to ignore the missing data when doing the calculation. For example if a given catchment has a handful of observations that are showing as NaNs for a given year, the mean for that catchment-year will be calculated ignoring the NaN values completely. Only catchment-years where *all* the observations are NaN does a NaN get returned in an arithmetic calculation. This behaviour is customisable. Take a look at the [Pandas documentation](https://pandas.pydata.org/) for more information.

Pandas [groupby](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) logic is very powerful and useful, especially this ability to parse dates and times. This example is just one of many things that can be done with it. Another example could be to add a column describing a particular catchment property, like steepness or size, group the catchments into bins from smallest to largest or shallowest to steepest, and see how the mean flow varies with these properties.

Let's keep it simple and take a look at a couple of the annual averages we've got.

In [ ]:
annavgs['39001'].plot(label='Thames')
axes = plt.gca()
annavgs['23001'].plot(ax=axes, label='Tyne')
plt.legend()

corr = np.corrcoef(annavgs['39001'], annavgs['23001'])[0,1] #correlation between the two time series
plt.title('R = ' + str(corr))

The Tyne is a large river in the northeast of England. There is not as much correlation between the two rivers as you might think!

Pandas can also group by meteorological season, which are defined based on the calendar months:
- Winter: December, January, February (DJF)
- Spring: March, April, May (MAM)
- Summer: June, July, August (JJA)
- Autumn: September, October, November (SON)

We can create seasonal averages by making further use of pandas's date-time handling. Let's look at the 90th percentile of the winter flows in each year for the same two catchments above.

Note that because 'quarters starting in December' is not a property of the Index (unlike years), we have to use a more sophisticated method to group the rows the way we would like.

Pandas has a [```Grouper```](https://pandas.pydata.org/docs/reference/api/pandas.Grouper.html) function that allows for more complicated groupings. To see what else it allows, you can run ```pd.Grouper?```.

In [ ]:
pc90 = obsdata.groupby(pd.Grouper(freq='QS-DEC')).quantile(0.9)

pc90

To select out only the winter flows, we want every 4th row, which we can represent as ```::4```. This is python indexing shorthand for ```0:-1:4``` which translates as the 0th (first) row to the -1th (last) row with a 'stride' (or step) of 4.

You'll see [```loc```](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html) and [```iloc```](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iloc.html) a lot with Pandas. These are powerful methods used to select out rows and columns from the table.[```loc```](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html) can be used to select out cells or ranges of cells using the row and column *names* or *labels*, whereas [```iloc```](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iloc.html) is used to select based on the row and column *indices* (e.g. the cell at row 8 and column 5 would be selected with ```obsdata.iloc[8, 5]``` and ```obsdata.loc['1961-01-08', '04003']``` - try it out for yourself!).

In [ ]:
pc90_DJF = pc90.iloc[::4, :]
pc90_DJF

In [ ]:
thames = pc90_DJF['39001'].plot(label='thames')
axes = plt.gca()
tyne = pc90_DJF['23001'].plot(ax=axes, label='tyne')
plt.legend()

corr = np.corrcoef(pc90_DJF['39001'], pc90_DJF['23001'])[0,1]
plt.title('R = ' + str(corr))

## [ASCII Grid files (ESRI format)](https://en.wikipedia.org/wiki/Esri_grid)

Another common file you might come across in hydrology is a file with the extension '**.grd**'. These files are often used as inputs/outputs of hydrological models. They are another textual representation of gridded data.

Ultimately these are just numbers laid out in a (usually) tab-separated pattern with some header lines at the top describing metadata needed to interpret the data into a grid. I find the easiest way of working with these is to read them into [Xarray](https://docs.xarray.dev/en/stable/), the now well established go-to library for working with gridded and N-dimensional data in various formats.

Here we'll develop the code to do this, but first let's take a look at what a typical grd file looks like if you open it in a text editor:

**Bonus:** The next code cell also shows how to display an image stored in an object store

In [ ]:
from PIL import Image
fs_img = s3fs.S3FileSystem(anon=True, endpoint_url="https://fdri-o.s3-ext.jc.rl.ac.uk")
display(Image.open(fs_img.open('s3://example-data/grdfile.png')))

Here you can clearly see the 6 header lines, followed by the rows & columns of numeric data.

- The **ncols** and **nrows** information is fairly self-explanatory
- The **xllcorner** and **yllcorner** are telling you what the x and y coordinates are of the 'lower left' corner of the grid. Important to note that these numbers represent the *corner* of the gridcell in the lower left corner of the grid, not it's centre point.
- **cellsize** tells you the x and y extent of the each cell in the grid, i.e. the distance between gridcells
- **NODATA_value** is also fairly self-explanatory

These files are only able to represent regular grids (grids that have consistent spacing between the grid cells) that also have the same spacing in the x and y direction.

The header information contains everything we need to know in order to read this into Xarray.

First we need to read in the original file. Reading in any textual data in python is simple:

In [ ]:
s3 = s3fs.S3FileSystem(anon=True, endpoint_url="https://fdri-o.s3-ext.jc.rl.ac.uk")
grdfile = s3.open('s3://example-data/absw_230_2009_06.grd')

In [ ]:
grdfile.readline()

The [```readline()```](https://docs.python.org/3/tutorial/inputoutput.html#methods-of-file-objects) function reads the current line as a string, then the next time it is called, does the same for the next line. To grab the entire file contents at once, we can use ```readlines()```:

In [ ]:
filecontents = grdfile.readlines()

filecontents

Each row becomes an item in the list

**Note:** That we seem to missing the top row of the file. This is because we already called ```readline``` which advances the line number the next call to ```readline``` or ```readlines``` will read from. To read from the top of the file, the file needs to be reopened.

When we have finished working with a file using this method it is important to manually close it, otherwise it will remain open in memory and can cause all sorts of interesting errors and bugs, especially when loops are involved!

In [ ]:
grdfile.close()

We could read in the entire file this way, however for numeric data other libraries, mainly [Numpy](https://numpy.org/), tend to be easier to work with. But now that we know how to read in text, we can pull out the header information which we can then use to construct the grid.

In [ ]:
filein = 's3://example-data/absw_230_2009_06.grd'
grdfile = s3.open(filein)
ncols  = int(grdfile.readline().split()[1])
nrows  = int(grdfile.readline().split()[1])
xllc   = float(grdfile.readline().split()[1])
yllc   = float(grdfile.readline().split()[1])
res    = float(grdfile.readline().split()[1])
nodata = float(grdfile.readline().split()[1])
print('ncols: ' + str(ncols))
print('nrows: ' + str(nrows))
print('xllc: ' + str(xllc))
print('yllc: ' + str(yllc))
print('res: ' + str(res))
print('nodata: ' + str(nodata))
grdfile.close()

Next we can use [Numpy's ```loadtxt```](https://numpy.org/doc/stable/reference/generated/numpy.loadtxt.html) function to read in the actual numeric data, skipping the header lines (skip the first 6 rows):

In [ ]:
grdfile = s3.open(filein)
data = np.loadtxt(grdfile,skiprows=6)
grdfile.close()

In [ ]:
data

Numpy has cleverly determined the data it is reading in is a grid, and stored it as such in an array

Now all we need to do is generate the coordinates from the information we read in from the header:

The [Numpy ```linspace```](https://numpy.org/doc/stable/reference/generated/numpy.linspace.html) command generates a sequence of equally spaced numbers: ```np.linspace(start, end, number of elements in sequence)```

We can work out all of these from the header information. Remember that the xllc and yllc are the coordinates of the *corner* of the lower left gridcell, and generally gridcell coordinates are for the *centre* of each gridcell.

Note the use of ```res/2```. This is because ```res``` defines the size of each grid cell and we want the center points so we have to move half the width and half the height from the bottom left corner.

In [ ]:
xcoords = np.linspace(xllc + (res/2), xllc + (res/2) + (res*(ncols-1)), ncols)
ycoords = np.linspace(yllc + (res/2), yllc + (res/2) + (res*(nrows-1)), nrows)

In [ ]:
xcoords

In [ ]:
ycoords

Now we are ready for Xarray to take charge. Xarray requires 3 key pieces of information:
- The data values themselves
- The names of the dimensions/coordinates (in this case they can just be 'x' and 'y')
- The coordinates of each of the dimensions (in this case the x and y coordinates we've just generated)

In [ ]:
ydimname = 'y'
xdimname = 'x'
grdxr = xr.DataArray(data, coords=[(ydimname, ycoords), (xdimname, xcoords)])
grdxr = grdxr.where(grdxr != nodata)

In [ ]:
grdxr

An Xarray DataArray is a coordinate-aware representation of our data, much like a version of Pandas designed for gridded and N-dimensional data, allowing operations and analyses to be carried out based on the *coordinate values*. We will learn more about Xarray soon!

For now, let's plot the data and see what it looks like:

In [ ]:
grdxr.plot()

Oh dear, that appears to be upside down! How might we fix this?

In [ ]:
xcoords = np.linspace(xllc + (res/2), xllc + (res/2) + (res*(ncols-1)), ncols)
ycoords = np.linspace(yllc + (res/2), yllc + (res/2) +  (res*(nrows-1)), nrows)[::-1] # using -1 inverts the y coordinates

ydimname = 'y'
xdimname = 'x'
grdxr = xr.DataArray(data, coords=[(ydimname, ycoords), (xdimname, xcoords)])
grdxr = grdxr.where(grdxr != nodata)

In [ ]:
grdxr.plot()

Now we have a basic Xarray representation of the data, which comes with various powerful analysis tools, some of which we will explore in the third notebook, with data that is a little more interesting than all zeroes!

## Further Resources

The interested reader might want to check out [Polars](https://pola.rs/), which is a more efficient rewrite of pandas for improved speed in handling large tabular datasets and databases. The commands are largely the same as Pandas.

- [Pandas documentation](https://pandas.pydata.org/docs/user_guide/index.html)
- [Numpy loadtxt documentation](https://numpy.org/doc/stable/reference/generated/numpy.loadtxt.html)
- [Pandas groupby documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html)
- [Pandas Grouper documentation](https://pandas.pydata.org/docs/reference/api/pandas.Grouper.html)
- [National River Flow Archive (NRFA) catchment information](https://nrfa.ceh.ac.uk/data/about-data/catchment-information)